In [27]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

class neuralNetwork:

    def buildLayer(_inputTensor, _numUnits):
        '''
        input:
            _inputTensor S = θ(Xprev)from the prev layer
            _numUnits is the num of neurons in this layer
    
        intermediate xavierInit:
            W is initialized as Xavier
            W is input.shape[1] by num units
    
        output:
        weighted sum of inputs
    
        '''
    
        # zero mean independent Gaussians whose variance is 3/(#input + #outputs)

    
        dim1 = _inputTensor.get_shape().as_list()[1]
    
        #Xavier
        #W = tf.get_variable("W", shape = [dim1, _numUnits], initializer=tf.contrib.layers.xavier_initializer())
        variance = 3.0 / (dim1 + _numUnits)
        W = tf.variable(tf.truncated_normal(shape = [dim1, _numUnits],  stddev = math.sqrt(variance)))
        b = tf.Variable(0.0, name='biases')
    
        # reduce mean so that each neuron has one output
        #print("W", W)
        X = tf.matmul(tf.transpose(W), tf.cast(_inputTensor, dtype = tf.float_32) + b
    
        #print("X", X)

        S = tf.nn.relu(X)
    
        return S

    def buildNet(_data, _numUnits):
        '''
        input: 
            number of hidden units
            data
        output:
            predicted labels
        '''
    
        return buildLayer(_data, numUnits)
    

    
    
class loadData:
    def __init__(self):
        self.flatten = True
        self.addOnes = False
        
        self.data_path = '/Users/vikuo/Documents/GitHub/ece521/assi/A3/notMNIST.npz'
    def arrFlatten(self, arr):
        '''
        type np array
        '''
        dataDim1, dum1, dum2 = arr.shape
        dum_sq = dum1 * dum2
        arr = np.reshape(arr, [ dataDim1 ,dum_sq ])
        return arr   
    def convertTarget(self, targetValues):
        numClasses = np.max(targetValues) + 1
        return np.eye(numClasses)[targetValues]

    def loadNotMnits(self):
        with np.load(self.data_path) as data:
            Data, Target = data ["images"], data["labels"]
            np.random.seed(521)
            randIndx = np.arange(len(Data))
            np.random.shuffle(randIndx)
            Data = Data[randIndx]/255.
            
            if self.flatten:
                Data = self.arrFlatten(Data)
            
            Target = Target[randIndx]
            trainData, trainTarget = Data[:15000], Target[:15000]
            validData, validTarget = Data[15000:16000], Target[15000:16000]
            testData, testTarget = Data[16000:], Target[16000:]
            
            trainTarget = self.convertTarget(trainTarget)
            validTarget = self.convertTarget(validTarget)
            testTarget = self.convertTarget(testTarget)
 
        return trainData, trainTarget, validData, validTarget, testData, testTarget

    
    
if __name__ == '__main__':  


    tf.reset_default_graph()
    init = tf.global_variables_initializer()
    sess = tf.InteractiveSession()
    sess.run(init)
    

    

TypeError: unhashable type: 'list'